# Proyek Analisis Data: [Input Nama Dataset]
- **Nama:** [Input Nama]
- **Email:** [Input Email]
- **ID Dicoding:** [Input Username]

## Menentukan Pertanyaan Bisnis

- Pertanyaan 1
- Pertanyaan 2

## Import Semua Packages/Library yang Digunakan

In [69]:
import pandas as pd

## Data Wrangling

### Gathering Data

**Read Dataset**

In [70]:
df = pd.read_csv("data/hour.csv")
df.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


**Dataset Information**

In [71]:
print("Rows:", df.shape[0])
print("Columns:", df.shape[1])

Rows: 17379
Columns: 17


**Check Missing Values**

In [72]:
print("Missing Values")
print(df.isnull().sum())

Missing Values
instant       0
dteday        0
season        0
yr            0
mnth          0
hr            0
holiday       0
weekday       0
workingday    0
weathersit    0
temp          0
atemp         0
hum           0
windspeed     0
casual        0
registered    0
cnt           0
dtype: int64


**Check Duplicated Data**

In [73]:
print("Duplicated Data:", df.duplicated().sum())

Duplicated Data: 0


**Check Data Types**

In [74]:
print("Data Types")
print(df.dtypes)

Data Types
instant         int64
dteday         object
season          int64
yr              int64
mnth            int64
hr              int64
holiday         int64
weekday         int64
workingday      int64
weathersit      int64
temp          float64
atemp         float64
hum           float64
windspeed     float64
casual          int64
registered      int64
cnt             int64
dtype: object


**Insight:**
- xxx
- xxx

### Assessing Data

**Check Outliers**

In [78]:
Q1 = df["cnt"].quantile(0.25)
Q3 = df["cnt"].quantile(0.75)
IQR = Q3 - Q1

maximum = Q3 + 1.5 * IQR
minimum = Q1 - 1.5 * IQR

lower_than = df["cnt"] < minimum
higher_than = df["cnt"] > maximum

outliers = df[lower_than | higher_than]

print("Outliers:", outliers.shape[0])

Outliers: 134


**Insight:**
- xxx
- xxx

### Cleaning Data

**Remove Outliers**

In [ ]:
df.drop(df[lower_than].index, inplace=True)
df.drop(df[higher_than].index, inplace=True)

**Dataset Information after Outliers**

In [ ]:
print("Rows: ", df.shape[0])
print("Columns: ", df.shape[1])

Rows:  16874
Columns:  17


**Insight:**
- xxx
- xxx

## Exploratory Data Analysis (EDA)

### Explore ...

**Insight:**
- xxx
- xxx

## Visualization & Explanatory Analysis

### Pertanyaan 1:

### Pertanyaan 2:

**Insight:**
- xxx
- xxx

## Analisis Lanjutan (Opsional)

## Conclusion

- Conclution pertanyaan 1
- Conclution pertanyaan 2